<a href="https://colab.research.google.com/github/pinilDissanayaka/Wikipedia-RAG-Application/blob/main/Notebook.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 61.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 227.1/227.1 kB 25.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 214.4/214.4 kB 24.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 374.2/374.2 kB 31.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 71.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 431.4/431.4 kB 46.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.7/274.7 kB 32.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 63.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 77.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━

In [2]:
import os
from pinecone import Pinecone, ServerlessSpec
from google.colab import userdata
from langchain.document_loaders import WikipediaLoader
from langchain_groq import ChatGroq
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone as PineconeStore
from langchain.prompts import PromptTemplate
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

In [3]:
query="India"

In [4]:
loader=WikipediaLoader(query=query)
docs=loader.load()

docs[0]

Document(metadata={'title': 'India', 'summary': "India, officially the Republic of India (ISO: Bhārat Gaṇarājya), is a country in South Asia.  It is the seventh-largest country by area; the most populous country as of June 2023; and from the time of its independence in 1947, the world's most populous democracy. Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west; China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is in the vicinity of Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime border with Thailand, Myanmar, and Indonesia.\nModern humans arrived on the Indian subcontinent from Africa no later than 55,000 years ago.\nTheir long occupation, initially in varying forms of isolation as hunter-gatherers, has made the region highly diverse, second only to Africa in human genetic diversity. Settl

In [33]:
textChunks=RecursiveCharacterTextSplitter(chunk_size=1600,
                                          chunk_overlap=200,
                                          keep_separator=False).split_documents(docs)

textChunks[0]

Document(metadata={'title': 'India', 'summary': "India, officially the Republic of India (ISO: Bhārat Gaṇarājya), is a country in South Asia.  It is the seventh-largest country by area; the most populous country as of June 2023; and from the time of its independence in 1947, the world's most populous democracy. Bounded by the Indian Ocean on the south, the Arabian Sea on the southwest, and the Bay of Bengal on the southeast, it shares land borders with Pakistan to the west; China, Nepal, and Bhutan to the north; and Bangladesh and Myanmar to the east. In the Indian Ocean, India is in the vicinity of Sri Lanka and the Maldives; its Andaman and Nicobar Islands share a maritime border with Thailand, Myanmar, and Indonesia.\nModern humans arrived on the Indian subcontinent from Africa no later than 55,000 years ago.\nTheir long occupation, initially in varying forms of isolation as hunter-gatherers, has made the region highly diverse, second only to Africa in human genetic diversity. Settl

In [7]:
os.environ['PINECONE_API_KEY'] = userdata.get('PINECORN_API_KEY')
os.environ['GROQ_API_KEY'] = userdata.get('GROQ_API_KEY')

In [8]:
pinecone=Pinecone(api_key=os.environ['PINECONE_API_KEY'])

In [9]:
print('Available Indexes at Pinecone :')
print(f"\t {pinecone.list_indexes().names()}")

Available Indexes at Pinecone :
	 ['american-history', 'pdf-rag', 'time-table']


In [25]:
INDEX_NAME='wiki'
DIMENSIONS=768
EMBEDDING_MODEL='all-mpnet-base-v2'

In [27]:
if not INDEX_NAME in pinecone.list_indexes().names():
  pinecone.create_index(INDEX_NAME,
                        dimension=DIMENSIONS,
                        metric='cosine',
                        spec=ServerlessSpec(cloud='aws',
                                            region='us-east-1'))
else:
  print('Index already exists')

In [24]:
embeddings=HuggingFaceEmbeddings(model_name=EMBEDDING_MODEL,
                                 show_progress=True,
                                 model_kwargs = {'device': 'cpu'},
                                  encode_kwargs = {'normalize_embeddings': False})

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/571 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [35]:
vectorStore=PineconeStore.from_documents(textChunks,
                                          embeddings,
                                          index_name=INDEX_NAME)

Batches:   0%|          | 0/3 [00:00<?, ?it/s]

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Wed, 24 Jul 2024 12:49:59 GMT', 'Content-Type': 'application/json', 'Content-Length': '115', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '948', 'x-pinecone-request-id': '7292369274819702787', 'x-envoy-upstream-service-time': '37', 'server': 'envoy'})
HTTP response body: {"code":3,"message":"Metadata size is 41497 bytes, which exceeds the limit of 40960 bytes per vector","details":[]}


In [ ]:
VectorStoreRetriever=vectorStore.as_retriever(search_type='similarity',
                                                search_kwargs={'k': 3})

keyworkSearchRetriever=BM25Retriever.from_documents(textChunks, k=3)

retriever=EnsembleRetriever(retrievers=[VectorStoreRetriever, keyworkSearchRetriever],
                            weights=[0.9, 0.2])

In [ ]:
promptTemplate = """Given the following context and a question, generate an answer based on this context only.
In the answer try to provide as much text as possible from "response" section in the source document context without making much changes.
If the answer is not found in the context, kindly state "I don't know." Don't try to make up an answer.

CONTEXT: {context}

QUESTION: {question}"""

prompt = PromptTemplate.from_template(promptTemplate)

print(prompt)


In [ ]:
llm = ChatGroq(
    temperature=0.8,
    model="llama3-70b-8192"
)

In [ ]:
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
response=chain.invoke('India')
print(response)